In [ ]:
import tkinter as tk
from tkinter import messagebox
import lib_scen_intersection as lsi
import lib_map_draw as dr
import lib_spawn as spb
import lib_traffic as trf
import lib_sensors as sen
import carla
from importlib import reload
reload(lsi)
reload(dr)
reload(spb)
reload(trf)
reload(sen)

class TrafficSimulatorGUI:
    def __init__(self, root, spacing=10):
        self.root = root
        self.root.title("Junction Scenarios")
        self.spacing = spacing
        self.world, self.crossings = self.connect_sim();
        
        self.spawns = None    #list of waypoint couples surround sleected junction
        self.gr_pairs = None  #dictionary 8map) of wp couples where the key is the common destination waypoint [1] = [(a,1), (b,1), (c,1)]
        self.crossing_index = 0
        self.spawn_index = 0

        self.ego_vehicle = None #(vehicle, wp)
        self.current_spawned = None
        self.npc_vehicles = []
        self.threads = []

        
        # sim reset button
        tk.Button(root, text="Reset Simulator", command=self.reset_simulator).grid(row=0, column=0, columnspan=4, pady=self.spacing)

        # init button for junction
        self.create_navigation_section("Junction", self.crossings,
                                       "Prev Junction", "Next Junction", 1)

        # Init button for spawn
        self.create_navigation_section("Spawn", self.spawns,
                                       "Prev Spawn", "Next Spawn", 2)

        # "Spawn vehicle" Button 
        tk.Button(root, text="Spawn vehicle", command=self.spawn_vehicle).grid(row=3, column=0, pady=self.spacing)
        tk.Button(root, text="Set Current Spawn as Ego", command=self.set_ego).grid(row=3, column=1, pady=self.spacing)
        tk.Button(root, text="Spawn on common routes", command=self.spawn_common_route).grid(row=3, column=2, padx=self.spacing)
        
        #tk.Label(root, text="BOT number:").grid(row=4, column=0, padx=self.spacing, pady=self.spacing)
        #self.traffic_entry = tk.Entry(root, width=8)  
        #self.traffic_entry.grid(row=4, column=1, padx=self.spacing, pady=self.spacing)

        tk.Button(root, text="Focus EGO", command=self.focus_ego).grid(row=4, column=0, padx=self.spacing, pady=self.spacing)
        # DRAW AND SPAWN ON COMMON ROUTES
        tk.Button(root, text="Draw common routes", command=self.draw_common_route).grid(row=4, column=1, padx=self.spacing)
        tk.Button(root, text="Fill Junction", command=self.fill_junction).grid(row=4, column=2, padx=self.spacing, pady=self.spacing)

        # Selection menu for ego agent profile
        tk.Label(root, text="Ego Agent Profile:").grid(row=5, column=0, padx=self.spacing, pady=self.spacing)
        self.spawn_profile = tk.StringVar(value="Select")
        spawn_menu = tk.OptionMenu(root, self.spawn_profile, "Cautious", "Normal", "Aggressive",
                                   command=self.display_spawn_profile)
        spawn_menu.grid(row=5, column=1, padx=self.spacing, pady=self.spacing)
        
        #checkbox wip
        self.ignore_traffic_lights = tk.BooleanVar(value=False)
        tk.Checkbutton(root, text="Skip Traffic Lights", variable=self.ignore_traffic_lights).grid(row=5, column=2, padx=self.spacing)

        # Selection menu for traffic agent profile
        tk.Label(root, text="Traffic Agent Profile:").grid(row=6, column=0, padx=self.spacing, pady=self.spacing)
        self.traffic_profile = tk.StringVar(value="Select")
        traffic_menu = tk.OptionMenu(self.root, self.traffic_profile, "Cautious", "Normal", "Aggressive",
                                     command=self.display_traffic_profile)
        traffic_menu.grid(row=6, column=1, padx=self.spacing, pady=self.spacing)
        
        #start simulation button
        tk.Button(root, text="Start simulation", command=self.start_simulation).grid(row=10, column=0, columnspan=4, pady=self.spacing)


    #handler for navigation spawn and junction navigation buttons
    def create_navigation_section(self, label, array, left_text, right_text, row):
        
        tk.Label(self.root, text=f"{label} attuale:").grid(row=row, column=0, padx=self.spacing, pady=self.spacing)
        
        #Fist boot
        if label == "Junction":
           display_label = tk.Label(self.root, text=f"{self.crossing_index + 1}/{len(self.crossings)}")
           
            # for each junction, retrive the list of surroinding waypoint couples. Each couple is indexed by self.crossing_index
           self.gr_pairs = lsi.group_pairs_by_second_waypoint(self.crossings[self.crossing_index][1].get_waypoints(carla.LaneType.Driving) )
           self.spawns = spb.list_all_behind_wp_couple(self.world, self.gr_pairs, 5)
           
           dr.focus_above(self.world, self.spawns[0][0].location, 60, -90)
           self.draw_common_route()
            
        elif label == "Spawn":
            display_label = tk.Label(self.root, text=f"{self.spawn_index + 1}/{len(self.spawns)}")

            #retrieve vehicle at current position
            self.current_spawned = spb.get_vehicle_at_spawn_point(self.spawns[self.spawn_index], self.npc_vehicles, 3)
            dr.focus_above(self.world, self.spawns[self.spawn_index][0].location, 60, -90)
            self.draw_common_route()
            
        display_label.grid(row=row, column=1, padx=self.spacing, pady=self.spacing)
        
        #function activated by navigation buttons
        def update_label(direction):
            nonlocal display_label
            if label == "Junction":
                dr.pair_counter = 0
                self.crossing_index = (self.crossing_index + direction) % len(self.crossings)
                display_label.config(text=f"{self.crossing_index  + 1 }/{len(self.crossings)}")#self.crossings[self.crossing_index][0]) #hunction contiene coppie id, coordinate
                
                # for each junction, retrive the list of surroinding waypoint couples. Each couple is indexed by self.crossing_index
                self.gr_pairs = lsi.group_pairs_by_second_waypoint(self.crossings[self.crossing_index][1].get_waypoints(carla.LaneType.Driving) )
                self.spawns = spb.list_all_behind_wp_couple(self.world, self.gr_pairs, 5)
                
                dr.focus_above(self.world, self.spawns[0][0].location, 60, -90)
                self.draw_common_route()
                self.spawn_index = 0
                
            elif label == "Spawn":
                self.spawn_index = (self.spawn_index + direction) % len(self.spawns)
                display_label.config(text=f"{self.spawn_index + 1}/{len(self.spawns)}")

                #retrieve vehicle at current position
                self.current_spawned = spb.get_vehicle_at_spawn_point(self.spawns[self.spawn_index], self.npc_vehicles, 3)
                dr.focus_above(self.world, self.spawns[self.spawn_index][0].location, 60, -90)
                self.draw_common_route()

        tk.Button(self.root, text=left_text, command=lambda: update_label(-1)).grid(row=row, column=2, padx=self.spacing, pady=self.spacing)
        tk.Button(self.root, text=right_text, command=lambda: update_label(1)).grid(row=row, column=3, padx=self.spacing, pady=self.spacing)
    
    #reset button and cleans simulator
    def reset_simulator(self):
        # reset indexes of junctions and spawn point
        self.crossing_index = 0
        self.spawn_index = 0

        self.ego_vehicle = None
        self.current_spawned = None
        self.npc_vehicles = []

        #self.traffic_entry.delete(0, tk.END)

        # Selection reset
        self.spawn_profile.set("Select")
        self.traffic_profile.set("Select")

        # Resetta lo stato della checkbox
        self.ignore_traffic_lights.set(False)

        #delete actors from sim
        lsi.reset_sim(self.world)

        # init button for junction
        self.create_navigation_section("Junction", self.crossings,
                                       "Prev Junction", "Next Junction", 1)
        # Init button for spawn
        self.create_navigation_section("Spawn", self.spawns,
                                       "Prev Spawn", "Next Spawn", 2)
        trf.stop_thread = True;
        
        messagebox.showinfo("Reset Completed", "All actors and sensors destroyed.")

    #BUTTON SPAWN EGO VEHICLE
    def spawn_vehicle(self):
        #traffic_lights_status = "Traffic Light ignored" if self.ignore_traffic_lights.get() else "Traffic Light observed"
        
        (v,w) = spb.spawn_vehicle(self.world, self.spawns[self.spawn_index])
        if v is not None:
           self.current_spawned =  (v,w)
           self.npc_vehicles.append((v,w))
           print(f"Spawned {v}")
        else: #get vehicle spawned at same location
           messagebox.showinfo("INFO", f"Current spawn location already busy by {self.current_spawned} len npclist: {len(self.npc_vehicles)}")
           
    #BUTTON SET EGO VEHICLE
    def set_ego(self):

        #check if there is a vehicle in current focused position
        if self.current_spawned is None:
           messagebox.showinfo("INFO", "No vehicle spawned at current location!!")
        else:
            if self.ego_vehicle is not None:
               self.npc_vehicles.append(self.ego_vehicle)
            self.ego_vehicle = self.current_spawned
            self.npc_vehicles = spb.recompute_npc_list(self.npc_vehicles, self.current_spawned)
            print(self.ego_vehicle[0])

    def focus_ego(self):
        if  self.ego_vehicle is not None:
            v, w_dest = self.ego_vehicle
            dr.focus_above(self.world, v.get_transform().location, 60, -90)
        else:
            messagebox.showinfo("INFO", "No ego vehicle set yet.")
        #dr.draw_route(self.world, spb.get_wp_from_vh(self.world, v), w_dest, '<->', 255, 255, 0, 300)
    
    #spawn vehicles on all junction edges
    def fill_junction(self):
        self.npc_vehicles += spb.spawn_traffic(self.world, self.spawns) 

    def display_spawn_profile(self, selection):
        print("Spawn agent profile", f"Profilo selezionato per Spwan: {selection}")

    def display_traffic_profile(self, selection):
        print("Traffic profile", f"Profilo selezionato per Traffico: {selection}")

    #SIM CONNECTION 
    def connect_sim(self):
        client = carla.Client('localhost', 2000)
        client.set_timeout(5.0)
        world = client.get_world()
        bp_lib = world.get_blueprint_library() 
        map = world.get_map()
        
        # wp list with 2 mt offset
        waypoints = map.generate_waypoints(2.0)
        
        # junction array (id, oggetto_incrocio)
        unique_junctions = []
        
        # retrieving junction by adjacent waypoints
        #!!: different waypoints may belong to same junction
        for waypoint in waypoints:
            if waypoint.is_junction:
                junction = waypoint.get_junction()
                if not(any(u[0] == junction.id for u in unique_junctions)):
                    unique_junctions.append( (junction.id, junction) )
        
        # Print junction number (9 distinct in town 10)
        print(f"Junctions found: {len(unique_junctions)}\n") 
        return (world, unique_junctions)
    
    #given current spawn and dest_wp, draws all routes starting from a spawn towards dest_wp
    def draw_common_route(self):
        wp2 = self.spawns[self.spawn_index][1]
        key_wp2 = (round(wp2.transform.location.x, 3), round(wp2.transform.location.y, 3))
        dr.draw_intersection(self.world, self.gr_pairs[key_wp2])
        dr.draw_spawn(self.world, self.spawns[self.spawn_index][0], 5, f"SP_{self.crossing_index + 1}_{self.spawn_index + 1}", 0, 255, 0)
        

    #spawns vehicle on all routes having the same wp as destination
    def spawn_common_route(self):
        wp2 = self.spawns[self.spawn_index][1]
        key_wp2 = (round(wp2.transform.location.x, 3), round(wp2.transform.location.y, 3))
        current_sp = self.spawns[self.spawn_index][0]
        
        for sp, wp_dest in self.spawns:
            wp_d = (round(wp_dest.transform.location.x, 3), round(wp_dest.transform.location.y, 3))
            if key_wp2 == wp_d:
               s,w = spb.spawn_vehicle(self.world, (sp, wp_dest))
               if s is not None:
                   self.npc_vehicles.append((s,w))

    ##### START SIMULATION FUNCTION AVVIO SIMULAZIONE #####
    def start_simulation(self):
        # Raccoglie i parametri selezionati
        crossing = self.crossings[self.crossing_index]
        spawn = self.spawns[self.spawn_index]
        #traffic_value = self.traffic_entry.get()
        spawn_profile = self.spawn_profile.get().lower()
        traffic_profile = self.traffic_profile.get().lower()
        traffic_lights_status = "Semafori ignorati" if self.ignore_traffic_lights.get() else "Semafori osservati"

        # Controlla siano settati i profili pena errore
        try:
            profiles = ["cautious", "normal", "aggressive"]
            msg = ""
            
            if spawn_profile not in profiles:
                msg = "Choose a driving profile for Ego vehicle!!"
                raise ValueError
            if len(self.npc_vehicles) != 0 and traffic_profile not in profiles:
                msg = "Choose a driving profile for Traffic!!"
                raise ValueError
            if self.ego_vehicle is None:
                msg = "No Ego vehicle spawned!!"
                raise ValueError
                
        except ValueError:
            messagebox.showinfo("ERROR: ", msg)
            return

        summary = (f"Simulation started with following parameters:\n"
                   f"- Selected Junction: {crossing}\n"
                   f"- Selected spawn: {spawn}\n"
                   #f"- Veicoli da generare: {traffic_value}\n"
                   f"- Agent ego profile: {spawn_profile}\n"
                   f"- Agent traffic profile: {traffic_profile}\n"
                   f"- Traffic light: {traffic_lights_status}")
        messagebox.showinfo("Start Simulation", summary)

        threads = []
        #START TRAFFIC IF PRESENT
        if len(self.npc_vehicles) != 0:
            self.threads += trf.start_vehicle_list(self.world, self.npc_vehicles, 'NPC', visible = 1, behaviour = traffic_profile) 
         #START EGO VEHICLE SPAWNED AT self.spawns[self.spawn_index] self.ego_vehicle
        self.threads.append( trf.start_vehicle(self.world, self.ego_vehicle, 'EGO_VEHICLE', visible = 2, behaviour = spawn_profile) )
        #EGO VEHICLE SENSORS
        end_route, dist_check, col_check = sen.attach_sensors(self.world, self.ego_vehicle[0], 30, 10) #last two: max_dist, tol_dist

        print(f"Sim results, End_route {end_route}, distanza rispettata {dist_check}, collision {col_check}")
        
        self.npc_vehicles = []
        self.ego_vehicle = None #(vehicle, wp)
        self.current_spawned = None

# Start widget
root = tk.Tk()
app = TrafficSimulatorGUI(root, spacing=7)
root.mainloop()


Junctions found: 9

Spawned Actor(id=733, type=vehicle.lincoln.mkz_2020)
Actor(id=733, type=vehicle.lincoln.mkz_2020)
Actor(id=737, type=vehicle.chevrolet.impala)
Spawned Actor(id=742, type=vehicle.lincoln.mkz_2020)
Actor(id=742, type=vehicle.lincoln.mkz_2020)
Actor(id=746, type=vehicle.bmw.grandtourer)
Actor(id=747, type=vehicle.mercedes.coupe_2020)
Actor(id=748, type=vehicle.dodge.charger_police_2020)
Actor(id=749, type=vehicle.dodge.charger_police_2020)
Spawn agent profile Profilo selezionato per Spwan: Aggressive
Traffic profile Profilo selezionato per Traffico: Aggressive
in start (<carla.libcarla.Vehicle object at 0x000002A63F22EF10>, <carla.libcarla.Waypoint object at 0x000002A6444F1150>)
Il veicolo Actor(id=742, type=vehicle.lincoln.mkz_2020) ha raggiunto la sua destinazione (Stop forzato=False)
Il veicolo Actor(id=743, type=vehicle.micro.microlino) ha raggiunto la sua destinazione (Stop forzato=False)
Il veicolo Actor(id=747, type=vehicle.mercedes.coupe_2020) ha raggiunto la s